In [ ]:
%load_ext autoreload
%autoreload 2

from utils.boolrank import DualEncoderModel
from utils.evaluation import evaluate_on_generated

model_name = 'BAAI/bge-small-en-v1.5'
# model_name = 'BAAI/llm-embedder'
# model_name = 'dmis-lab/biobert-v1.1'
model = DualEncoderModel(model_name)
model_name = model.model_name.split("/")[-1]
_ = model.eval()

In [ ]:
from utils.data_processing import paths_to_dataset

bool_key = "bool_query"
nl_key = "nl_query"
data_path = "training"
path = "query_data/{}.jsonl"
pubmed = path.format(data_path)
TAR = path.format("TAR_data")
sysrev = path.format("sysrev_conv")
dataset = paths_to_dataset([pubmed, TAR, sysrev], test_only_sources=["TAR", "sysrev"], test_only=True)
# dataset = paths_to_dataset([TAR, sysrev], test_only_sources=["TAR", "sysrev"], test_only=True)
dataset

In [ ]:
from utils.evaluation import evaluate

def generate_plots(format="pdf"):
    for key, data in dataset["test"].items():
        res = evaluate(model, data[bool_key], data[nl_key], plot=True, title=key)
        res["plot1"].savefig(f"models/clip/{model_name}/{key}1.{format}", format=format)
        res["plot2"].savefig(f"models/clip/{model_name}/{key}2.{format}", format=format)

In [ ]:
data = dataset["test"]["TAR"]
logits = evaluate(model, data[bool_key], data[nl_key])['logits']

In [ ]:
logs = model(data[bool_key][15], data[nl_key][:30])['logits']
(-logs).argsort()

In [ ]:
print(data[1])
print((-logits[15]).argsort())
ahead = [24, 20, 25, 22, 15]
for d in ahead:
    print(data[d])

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Input data (from your prints)

# Ranking info
ahead = [24, 20, 25, 22, 15]  # 15 should be correct but ranked too low

cutoff = 100
# Build dataframe
rows = []
for idx in ahead:
    rows.append([data[idx]["nl_query"][:cutoff]])

df = pd.DataFrame(rows, columns=["NL Query"])

# Plot as table
fig, ax = plt.subplots(figsize=(10, 2))
ax.axis("off")

table = ax.table(
    cellText=df.values,
    colLabels=df.columns,
    cellLoc="left",
    loc='center',
    colColours=["lightgrey"]*3,
)

# Color rows by status
for i in range(1, len(df) + 1):
    if i == 5:
        color = "lightgreen"
    else:
        color = "lightcoral"
    table[(i, 0)].set_facecolor(color)

# plt.tight_layout()
plt.title('Bool Query: (' + data[15]["bool_query"][:cutoff] + '...)')
plt.savefig("nl_bool_query_ranking.pdf", format="pdf")
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

cutoff = 100
# Build dataframe
rows = []
for idx in [1]:
    rows.append([data[idx]["nl_query"][:cutoff]])

df = pd.DataFrame(rows, columns=["NL Query"])

# Plot as table
fig, ax = plt.subplots(figsize=(10, 2))
ax.axis("off")

table = ax.table(
    cellText=df.values,
    colLabels=df.columns,
    cellLoc="left",
    loc='center',
    colColours=["lightgrey"]*3,
)

color = "lightgreen"
table[(1, 0)].set_facecolor(color)

# plt.tight_layout()
plt.title('Bool Query: (' + data[1]["bool_query"][:cutoff] + '...)')
plt.savefig("nl_bool_query_ranking2.pdf", format="pdf")
plt.show()

In [ ]:
# path = r"models\clip\biobert-v1.1\b4_lr8E-06_(pubmed-que_pubmed-sea_raw-jsonl)^2no[]\model.safetensors"
# model.load(path)
# generate_plots()

## Untrained

In [ ]:
evaluate_on_generated(model)

In [ ]:
evaluate_on_generated(model, ["id"])

## Model 1

In [ ]:
model.load(r"models\clip\bge-small-en-v1.5\b16_lr1E-05_(pubmed-que_pubmed-sea_raw-jsonl)^4\checkpoint-11288\model.safetensors")
# model.load(r"models\clip\biobert-v1.1\b16_lr1E-05_(pubmed-que_pubmed-sea_raw-jsonl)^4\checkpoint-14110\model.safetensors")
evaluate_on_generated(model)

In [ ]:
evaluate_on_generated(model, ["id"])

## Model 2

In [ ]:
model.load(r"models\clip\bge-small-en-v1.5\b16_(pubmed-que_pubmed-sea_raw-jsonl)^4\checkpoint-11288\model.safetensors")
# evaluate_on_generated(model)

In [ ]:
evaluate_on_generated(model, ["id"])